<a href="https://colab.research.google.com/github/tchotaneu/Detection_of_faults_in_high_voltage_electrical_transport_equipment_through_artificial_vision/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Detection of faults in high voltage electrical transport equipment through artificial visio**

**Connect to colab and  google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Jun  3 03:43:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 15420 (delta 1), reused 6 (delta 1), pack-reused 15413
Receiving objects: 100% (15420/15420), 14.07 MiB | 17.94 MiB/s, done.
Resolving deltas: 100% (10358/10358), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [ ]:
!echo "disque_casse" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-05-21 15:36:36--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  45.0MB/s    in 3.8s    

2022-05-21 15:36:40 (40.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

We zipped our images and labels into a zip file

1.   labels are files .txt
2.   images are files .jpg




In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/1.txt          
  inflating: data/obj/2.jpeg         
  inflating: data/obj/2.txt          
  inflating: data/obj/3.jpeg         
  inflating: data/obj/3.txt          
  inflating: data/obj/4.jpeg         
  inflating: data/obj/4.txt          
  inflating: data/obj/5.jpeg         
  inflating: data/obj/5.txt          
  inflating: data/obj/6.jpeg         
  inflating: data/obj/6.txt          
  inflating: data/obj/7.jpeg         
  inflating: data/obj/7.txt          
  inflating: data/obj/8.jpg          
  inflating: data/obj/8.txt          
  inflating: data/obj/9.jpg          
  inflating: data/obj/9.txt          
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.jpg         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.JPG         
  inflating: data/obj/12.txt         
  inflating: data/obj/13.JPG         
  inflating: data/obj/13.txt         
  inflating: 

In [ ]:
.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.461000 0.468500 0.032667 0.131000

['0', '0.461000', '0.468500', '0.032667', '0.131000']
0 data/obj/398.txt
0 0.461000 0.468500 0.032667 0.131000
0 0.527667 0.509500 0.111333 0.155000

['0', '0.527667', '0.509500', '0.111333', '0.155000']
1 data/obj/370.txt
0 0.527667 0.509500 0.111333 0.155000
0 0.542817 0.510876 0.271510 0.235600

['0', '0.542817', '0.510876', '0.271510', '0.235600']
2 data/obj/10.txt
0 0.542817 0.510876 0.271510 0.235600
0 0.451684 0.476562 0.340869 0.314827

['0', '0.451684', '0.476562', '0.340869', '0.314827']
3 data/obj/465.txt
0 0.451684 0.476562 0.340869 0.314827
0 0.525498 0.353933 0.100746 0.063670

['0', '0.525498', '0.353933', '0.100746', '0.063670']
4 data/obj/109.txt
0 0.525498 0.353933 0.100746 0.063670
0 0.439938 0.514960 0.163564 0.125665

['0', '0.439938', '0.514960', '0.163564', '0.125665']
5 data/obj/477.txt
0 0.439938 0.514960 0.163564 0.125665
0 0.490637 0.157805 0.154494 0.022699

['0', '0.490637', '0.157805', '0.154494', '0.022699']
6 data/o

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/313.jpg', 'data/obj/387.jpg', 'data/obj/287.jpg', 'data/obj/401.jpg', 'data/obj/417.jpg', 'data/obj/268.jpg', 'data/obj/346.jpg', 'data/obj/317.jpg', 'data/obj/263.jpg', 'data/obj/339.jpg', 'data/obj/299.jpg', 'data/obj/280.jpg', 'data/obj/198.jpg', 'data/obj/369.jpg', 'data/obj/288.jpg', 'data/obj/421.jpg', 'data/obj/333.jpg', 'data/obj/11.jpg', 'data/obj/362.jpg', 'data/obj/148.jpg', 'data/obj/271.jpg', 'data/obj/335.jpg', 'data/obj/324.jpg', 'data/obj/322.jpg', 'data/obj/302.jpg', 'data/obj/391.jpg', 'data/obj/304.jpg', 'data/obj/315.jpg', 'data/obj/191.jpg', 'data/obj/332.jpg', 'data/obj/330.jpg', 'data/obj/325.jpg', 'data/obj/344.jpg', 'data/obj/327.jpg', 'data/obj/405.jpg', 'data/obj/294.jpg', 'data/obj/318.jpg', 'data/obj/357.jpg', 'data/obj/259.jpg', 'data/obj/307.jpg', 'data/obj/406.jpg', 'data/obj/419.jpg', 'data/obj/286.jpg', 'data/obj/186.jpg', 'data/obj/331.jpg', 'data/obj/297.jpg', 'data/obj/358.jpg', 'data/obj/420.jpg', 'data/obj/312.jpg', 'data/obj/359.jpg', 

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.693230), count: 2, class_loss = 0.323381, iou_loss = 0.097690, total_loss = 0.421071 
 total_bbox = 119358, rewritten_bbox = 0.006703 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000016, iou_loss = 0.000000, total_loss = 0.000016 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.830851), count: 3, class_loss = 0.023613, iou_loss = 0.080025, total_loss = 0.103638 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.805996), count: 1, class_loss = 0.000660, iou_loss = 0.097747, total_loss = 0.098407 
 total_bbox = 119362, rewritten_bbox = 0.006702 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.828508), count: 2, class_loss = 0.017485, iou_loss = 0.042644, t

# Nouvelle section